In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#-------------------Inputs-------------------------------------------------------------
input_file = "Shear_and_Moment_4 Paddlers.csv" # or "Shear_and_Moment_2 Paddlers.csv"
inner_file = "Inner Hull.csv"
outer_file = "Outer Hull.csv"
#--------------------------------------------------------------------------------------

script_dir = os.path.abspath('')
file_path = os.path.join(script_dir, input_file)
info = pd.read_csv(file_path, header=None)

file_path = os.path.join(script_dir, inner_file) # file path for the text file input
in_coor = pd.read_csv(file_path, header=None)

file_path = os.path.join(script_dir, outer_file) # file path for the text file input
out_coor = pd.read_csv(file_path, header=None)

#x_coor = coordinates[0].iloc[1:] # get rid of [0,0,0]
x_in = in_coor[0]
y_in = in_coor[1]
z_in = in_coor[2]

x_out = out_coor[0]
y_out = out_coor[1]
z_out = out_coor[2]

station_no = x_out.nunique() # number of stations
station_out = x_out.unique() # location of each station
station_in = x_in.unique()


In [17]:
# Positive Bending Moment
value = float(info[1][3])
width = float(info[3][3])
depth = float(info[4][3])
target_station = float(info[2][3])

# Extract Station Information
Y1_in = []; Z1_in = []
Y2_in = []; Z2_in = []
Y1_out = []; Z1_out = []
Y2_out = []; Z2_out = []

for k in range(1,station_no-2,1):
    if k == target_station:
        X1_in = station_in[k]
        X2_in = station_in[k+1]
        X1_out = station_out[k]
        X2_out = station_out[k+1]
        
        for i in range(0,len(x_out),1):
            if x_out[i] == X1_out:
                Y1_out.append(y_out[i])
                Z1_out.append(z_out[i])

            elif x_out[i] == X2_out:
                Y2_out.append(y_out[i])
                Z2_out.append(z_out[i])
            
            if x_in[i] == X1_in:
                Y1_in.append(y_in[i])
                Z1_in.append(z_in[i])

            elif x_in[i] == X2_in:
                Y2_in.append(y_in[i])
                Z2_in.append(z_in[i])

# Find y_bar
volume = 0
product = 0 # y_bar*Volume
for i in range(0,len(Y1_out)-1,1):
    z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
    x = (abs(X1_in-X2_in) + abs(X1_out-X2_out))/2
    y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
    z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
    volume_i = x*y*z
    volume = volume + volume_i
    product = product + volume_i*z_i
    
y_bar_top = abs(product/volume)
y_bar_bottom = depth - y_bar_top

# Find I
I = 0
for i in range(0,len(Y1_out)-1,1):
    z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
    y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
    z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
    I_i = (1/12*y*z**3 + y*z*(abs(z_i) - y_bar_top)**2)/(10**(12)) # [m^4]
    I = I + I_i

stress_top = value*(y_bar_top/1000)/I
stress_bottom = value*(y_bar_bottom/1000)/I

print('Positive moment')
print('Compressive Flexural Stress = ', stress_top/10**6, ' MPa')
print('Tensile Flexural Stress = ', stress_bottom/10**6, ' MPa')


Positive moment
Compressive Flexural Stress =  0.776012126013614  MPa
Tensile Flexural Stress =  0.9270242324654842  MPa


In [18]:
# Negative Bending Moment
value = float(info[1][4])
width = float(info[3][4])
depth = float(info[4][4])
target_station = float(info[2][4])

# Extract Station Information
Y1_in = []; Z1_in = []
Y2_in = []; Z2_in = []
Y1_out = []; Z1_out = []
Y2_out = []; Z2_out = []

for k in range(1,station_no-2,1):
    if k == target_station:
        X1_in = station_in[k]
        X2_in = station_in[k+1]
        X1_out = station_out[k]
        X2_out = station_out[k+1]
        
        for i in range(0,len(x_out),1):
            if x_out[i] == X1_out:
                Y1_out.append(y_out[i])
                Z1_out.append(z_out[i])

            elif x_out[i] == X2_out:
                Y2_out.append(y_out[i])
                Z2_out.append(z_out[i])
            
            if x_in[i] == X1_in:
                Y1_in.append(y_in[i])
                Z1_in.append(z_in[i])

            elif x_in[i] == X2_in:
                Y2_in.append(y_in[i])
                Z2_in.append(z_in[i])

print("SHOULD BE GREATER THAN 1: " + str(len(Y1_out)))

# Find y_bar
volume = 0
product = 0 # y_bar*Volume
for i in range(0,len(Y1_out)-1,1):
    z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
    x = (abs(X1_in-X2_in) + abs(X1_out-X2_out))/2
    y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
    z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
    print("x: " + x + "\ny: " + y + "\nz: " + z)
    volume_i = x*y*z
    volume += volume_i
    product = product + volume_i*z_i
    
y_bar_top = abs(product/volume)
y_bar_bottom = depth - y_bar_top

# Find I
I = 0
for i in range(0,len(Y1_out)-1,1):
    z_i = ((Z1_out[i]+Z1_out[i+1]+Z1_in[i]+Z1_in[i+1])/4 + (Z2_out[i]+Z2_out[i+1]+Z2_in[i]+Z2_in[i+1])/4) /2
    y = (abs(Y1_in[i] - Y1_out[i]) + abs(Y1_in[i+1]-Y1_out[i+1]) + abs(Y2_in[i] - Y2_out[i]) + abs(Y2_in[i+1]-Y2_out[i+1]))/4
    z = (abs(Z1_out[i+1]-Z1_out[i]) + abs(Z1_in[i+1]-Z1_in[i]) + abs(Z2_out[i+1]-Z2_out[i]) + abs(Z2_in[i+1]-Z2_in[i]))/4
    I_i = (1/12*y*z**3 + y*z*(abs(z_i) - y_bar_top)**2)/10**(12) # [m^4]
    I = I + I_i

stress_top = value*(y_bar_top/1000)/I
stress_bottom = value*(y_bar_bottom/1000)/I

print('Negative moment')
print('Tensile Flexural Stress = ', stress_top/10**6, ' MPa')
print('Compressive Flexural Stress = ', stress_bottom/10**6, ' MPa')


SHOULD BE GREATER THAN 1: 0


ZeroDivisionError: division by zero